回顾多层感知机

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20) # 2 * 20
net(X)

tensor([[-0.1620, -0.0793,  0.1431,  0.0673,  0.0966,  0.1359, -0.1023, -0.0286,
          0.0487, -0.1550],
        [-0.1697, -0.0233,  0.1816,  0.0134,  0.0471,  0.0944, -0.2029,  0.0183,
          0.1670, -0.1843]], grad_fn=<AddmmBackward0>)

自定义块

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.0184,  0.1097,  0.0730,  0.0487, -0.0836,  0.2052, -0.0275,  0.3346,
          0.1413,  0.1403],
        [-0.0284,  0.0834,  0.1159,  0.0303, -0.1578,  0.1395,  0.0090,  0.2734,
          0.1115,  0.0310]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
            return X
net = MySequential(nn.Linear(20, 256),nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 3.0204e-01, -3.7004e-01, -4.7854e-01, -1.4433e-02, -3.4079e-01,
          2.3749e-01, -1.7188e-02,  2.8027e-01,  1.3271e-01,  1.8961e-01,
          2.3601e-01, -5.6924e-01,  6.1351e-01, -1.6358e-01,  2.7222e-01,
          6.1518e-04,  1.7524e-01, -4.7493e-01,  1.3230e-01, -1.9324e-01,
          3.4212e-01, -1.7250e-01, -7.5536e-02, -1.5026e-01, -4.0451e-01,
          2.0170e-01, -1.1454e-01, -1.6821e-01,  4.1968e-01,  2.0275e-01,
          3.2770e-02,  5.5130e-02, -7.7672e-01, -2.5013e-01,  2.0022e-02,
         -8.3509e-01,  4.3361e-01, -7.6710e-01, -9.8574e-01, -3.6373e-01,
          5.2517e-01, -3.8269e-01,  5.4766e-01,  3.1082e-01, -1.0424e-01,
         -4.9313e-01, -4.3185e-02, -2.5668e-01, -3.9637e-01, -4.0978e-01,
         -2.3987e-01,  2.9382e-01,  1.8648e-01, -4.8027e-01, -6.4770e-02,
          1.0662e-02,  7.9166e-01,  3.0165e-01, -7.0192e-01, -5.4201e-01,
         -2.8544e-01,  9.4480e-01,  7.1591e-01, -2.0591e-01,  6.2725e-01,
         -3.3718e-01, -2.8203e-01,  3.

在正向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        # weight不参加训练
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1612, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0682, grad_fn=<SumBackward0>)